In [2]:
# Imports
import tensorflow as tf
import numpy as np
import os, cv2, random, time, sys
import matplotlib.pyplot as plt
from keras.utils import to_categorical
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Dropout
from keras.models import Sequential
from sklearn.model_selection import train_test_split
from keras.callbacks import TensorBoard
import tensorboard
from keras import models
!pip install gradio
import gradio as gr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 MB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.2/320.2 kB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 88.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.2/168.2 kB 15.4 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [3]:
import os
import gdown
file = "1Pn_MiWhSuyA8qUpOeUosl_fa60nP4anN"
output = "COVID.zip"

if not os.path.exists(output):
  !gdown --fuzzy https://drive.google.com/us?id={file} -O {output}
else:
  print("File exists.")

Downloading...
From (original): https://drive.google.com/uc?id=1Pn_MiWhSuyA8qUpOeUosl_fa60nP4anN
From (redirected): https://drive.google.com/uc?id=1Pn_MiWhSuyA8qUpOeUosl_fa60nP4anN&confirm=t&uuid=c306bb06-0e67-487d-90da-9959e1bb96fc
To: /content/COVID.zip
100% 524M/524M [00:09<00:00, 53.3MB/s]


In [17]:
file = "1bodsVSr1cJCibbQsinLap-ietgMpzOZh"
output = "Fake_Generated_Images.zip"

if not os.path.exists(output):
  !gdown --fuzzy https://drive.google.com/us?id={file} -O {output}
else:
  print("File exists.")

Downloading...
From (original): https://drive.google.com/uc?id=1bodsVSr1cJCibbQsinLap-ietgMpzOZh
From (redirected): https://drive.google.com/uc?id=1bodsVSr1cJCibbQsinLap-ietgMpzOZh&confirm=t&uuid=71a85192-cfad-44ca-a58d-48a09eaecb51
To: /content/Fake_Generated_Images.zip
100% 219M/219M [00:03<00:00, 59.1MB/s]


In [ ]:
output_folder = "COVID"
if not os.path.exists(output_folder):
    # If the folder doesn't exist, unzip the file
    !unzip COVID.zip -d {output_folder}
    print(f"Unzipped to {output_folder}.")
else:
    print(f"The dataset folder already exists. Skipping unzip.")

In [18]:
output_folder = "Fake_Generated_Images"
if not os.path.exists(output_folder):
    # If the folder doesn't exist, unzip the file
    !unzip Fake_Generated_Images.zip -d {output_folder}
    print(f"Unzipped to {output_folder}.")
else:
    print(f"The dataset folder already exists. Skipping unzip.")

Archive:  Fake_Generated_Images.zip
  inflating: Fake_Generated_Images/fake_134.png  
  inflating: Fake_Generated_Images/fake_254.png  
  inflating: Fake_Generated_Images/fake_484.png  
  inflating: Fake_Generated_Images/fake_94.png  
  inflating: Fake_Generated_Images/fake_5.png  
  inflating: Fake_Generated_Images/fake_51.png  
  inflating: Fake_Generated_Images/fake_399.png  
  inflating: Fake_Generated_Images/fake_515.png  
  inflating: Fake_Generated_Images/fake_539.png  
  inflating: Fake_Generated_Images/fake_365.png  
  inflating: Fake_Generated_Images/fake_141.png  
  inflating: Fake_Generated_Images/fake_34.png  
  inflating: Fake_Generated_Images/fake_170.png  
  inflating: Fake_Generated_Images/fake_311.png  
  inflating: Fake_Generated_Images/fake_340.png  
  inflating: Fake_Generated_Images/fake_594.png  
  inflating: Fake_Generated_Images/fake_535.png  
  inflating: Fake_Generated_Images/fake_0.png  
  inflating: Fake_Generated_Images/fake_122.png  
  inflating: Fake_Gen

In [5]:
# Constants
DIRECTORY = 'COVID'
CLASSES = ['CT_NonCOVID', 'CT_COVID']
NUM_CLASSES = len(CLASSES)
IMG_H = 100
IMG_W = 100
IMG_C = 1
IMG_SHAPE = (IMG_H, IMG_W, IMG_C)
NUM_EPOCHS = 25
BATCH_SIZE = 64

In [6]:
# Preprocess the CT scan images
data = []
for c in CLASSES:
    folder = os.path.join(DIRECTORY, c)
    label = CLASSES.index(c)

    for img in os.listdir(folder):
        img_path = os.path.join(folder, img)

        if  '.DS_Store' not in img_path:
            img_arr = cv2.imread(img_path)
            img_arr = cv2.resize(img_arr, (IMG_H, IMG_W))
            img_arr = cv2.cvtColor(img_arr, cv2.COLOR_BGR2GRAY) # Convert image to grayscale
            data.append([img_arr, label])

random.shuffle(data)

X, y = [], []
for features, label in data:
    X.append(features)
    y.append(label)

# Normalize the data to range [0, 1]
X = np.array(X)
X = X / 255.0

# One-hot encoding of image labels
y = np.array(y)
y = to_categorical(y, NUM_CLASSES)

In [7]:
# Split data into training (70%), validation (10%), and testing (20%) datasets
X_remainder, X_test, y_remainder, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1000)
X_train, X_val, y_train, y_val = train_test_split(X_remainder, y_remainder, test_size = 0.125, random_state = 1000)

In [8]:
# Construct the CNN model
model = Sequential()
model.add(Conv2D(64, (3, 3), activation = 'relu', input_shape = IMG_SHAPE))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation = 'relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation = 'relu'))
model.add(Flatten())
model.add(Dense(128, activation = 'relu'))
model.add(Dense(NUM_CLASSES, activation = 'softmax'))

model.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 98, 98, 64)          │             640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 49, 49, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 47, 47, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 23, 23, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 21, 21, 128)         │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 56448)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │       7,225,472 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 2)                   │             258 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 7,447,810 (28.41 MB)

 Trainable params: 7,447,810 (28.41 MB)

 Non-trainable params: 0 (0.00 B)

In [9]:
# Set up the TensorBoard callback
tensorboard = TensorBoard(log_dir = f'COVID_prediction_{int(time.time())}')

In [10]:
# Compile and train the model on the training and validation datasets
model.compile(loss = 'categorical_crossentropy', metrics = ['accuracy'], optimizer = 'adam')
model.fit(X_train, y_train, epochs = NUM_EPOCHS, batch_size = BATCH_SIZE, validation_data = (X_val, y_val), callbacks = tensorboard)

Epoch 1/25
51/51 ━━━━━━━━━━━━━━━━━━━━ 151s 3s/step - accuracy: 0.6046 - loss: 0.7980 - val_accuracy: 0.7591 - val_loss: 0.4679
Epoch 2/25
51/51 ━━━━━━━━━━━━━━━━━━━━ 207s 3s/step - accuracy: 0.7887 - loss: 0.4359 - val_accuracy: 0.8065 - val_loss: 0.3661
Epoch 3/25
51/51 ━━━━━━━━━━━━━━━━━━━━ 196s 3s/step - accuracy: 0.8515 - loss: 0.3263 - val_accuracy: 0.8946 - val_loss: 0.2571
Epoch 4/25
51/51 ━━━━━━━━━━━━━━━━━━━━ 143s 3s/step - accuracy: 0.9032 - loss: 0.2318 - val_accuracy: 0.9355 - val_loss: 0.2074
Epoch 5/25
51/51 ━━━━━━━━━━━━━━━━━━━━ 207s 3s/step - accuracy: 0.9288 - loss: 0.1841 - val_accuracy: 0.9376 - val_loss: 0.1826
Epoch 6/25
51/51 ━━━━━━━━━━━━━━━━━━━━ 194s 3s/step - accuracy: 0.9385 - loss: 0.1490 - val_accuracy: 0.9441 - val_loss: 0.1513
Epoch 7/25
51/51 ━━━━━━━━━━━━━━━━━━━━ 139s 3s/step - accuracy: 0.9563 - loss: 0.1189 - val_accuracy: 0.9140 - val_loss: 0.2127
Epoch 8/25
51/51 ━━━━━━━━━━━━━━━━━━━━ 141s 3s/step - accuracy: 0.9679 - loss: 0.0856 - val_accuracy: 0.9441 - v

In [11]:
# Evaluate the model using the test dataset
results = model.evaluate(X_test, y_test, batch_size = BATCH_SIZE)
print('test_loss, test_accuracy:', results)

15/15 ━━━━━━━━━━━━━━━━━━━━ 12s 785ms/step - accuracy: 0.9454 - loss: 0.2682
test_loss, test_accuracy: [0.32173997163772583, 0.9430107474327087]


In [12]:
# Save the trained model
model.save(r'COMP496_CNN.keras')

In [13]:
# Backend functionality for Gradio interface
def make_prediction(img_arr):
    # Retrieve the model
    model = models.load_model(r'COMP496_CNN.keras')

    # Model makes prediction on image
    img_arr = cv2.resize(img_arr, (IMG_H, IMG_W))
    prediction = model.predict(img_arr.reshape(-1, IMG_H, IMG_W, IMG_C))

    # Classifications
    CLASSIFICATIONS = {
        'NonCOVID': 0,
        'COVID': 1
    }

    print(prediction)

    # Determine highest prediction
    highest = (-1 * sys.maxsize) - 1
    index = -1
    for i in range(len(prediction[0])):
        if prediction[0][i] > highest:
            highest = prediction[0][i]
            index = i

    for key in CLASSIFICATIONS:
        if CLASSIFICATIONS[key] == index:
            return key

In [21]:
import random
import os

# Directory path for the COVID images
covid_images_dir = "/content/Fake_Generated_Images"  # Path to the COVID images directory

# Load images dynamically from the specified directory
def load_images_from_directory(directory):
    try:
        images = [os.path.join(directory, img) for img in os.listdir(directory) if img.endswith((".png", ".jpg", ".jpeg"))]
        print(f"Loaded images from {directory}: {images}")  # Debugging line
        return images
    except Exception as e:
        print(f"Error loading images from {directory}: {e}")
        return []

# Load images from the COVID directory
covid_images = load_images_from_directory(covid_images_dir)

# Function to generate a random COVID image
def generate_image():
    try:
        # Select one random image from the COVID directory
        covid_image = random.choice(covid_images) if covid_images else None

        # Debugging line
        print(f"Selected COVID image: {covid_image}")

        # Ensure an image is available
        if not covid_image:
            raise ValueError("No images available in the COVID directory.")

        # Return the selected image for display in Gradio
        return covid_image
    except Exception as e:
        print(f"Error generating image: {e}")
        return None

# Create the Gradio app
with gr.Blocks() as demo:
    # CT Scan Generator Tab
    with gr.Tab("True Color GAN - MKBE GROUP 4"):
        gr.Markdown("### COVID-19 AI Generated Image")
        generate_button = gr.Button("Generate Image")
        image1 = gr.Image(label="COVID-19 AI generated image")
        generate_button.click(
            fn=generate_image,
            inputs=[],
            outputs=[image1],
        )
    # Tab for the image classification
    with gr.Tab("COVID-19 Prediction"):
        gr.Markdown("### Upload a CT Scan for Prediction")
        image_input = gr.Image(height=300, label="Upload CT Scan")
        output_label = gr.Label(num_top_classes=1, label="Prediction")
        predict_button = gr.Button("Predict")

        predict_button.click(
            fn=make_prediction,
            inputs=[image_input],
            outputs=[output_label],
        )
demo.launch(show_error=True, share=True)

Loaded images from /content/Fake_Generated_Images: ['/content/Fake_Generated_Images/fake_498.png', '/content/Fake_Generated_Images/fake_395.png', '/content/Fake_Generated_Images/fake_19.png', '/content/Fake_Generated_Images/fake_135.png', '/content/Fake_Generated_Images/fake_148.png', '/content/Fake_Generated_Images/fake_134.png', '/content/Fake_Generated_Images/fake_131.png', '/content/Fake_Generated_Images/fake_341.png', '/content/Fake_Generated_Images/fake_270.png', '/content/Fake_Generated_Images/fake_28.png', '/content/Fake_Generated_Images/fake_122.png', '/content/Fake_Generated_Images/fake_409.png', '/content/Fake_Generated_Images/fake_60.png', '/content/Fake_Generated_Images/fake_185.png', '/content/Fake_Generated_Images/fake_343.png', '/content/Fake_Generated_Images/fake_68.png', '/content/Fake_Generated_Images/fake_284.png', '/content/Fake_Generated_Images/fake_383.png', '/content/Fake_Generated_Images/fake_251.png', '/content/Fake_Generated_Images/fake_145.png', '/content/Fa

In [20]:
# Gradio interface
#demo = gr.Interface(fn = make_prediction, inputs = gr.Image(height = 300), outputs = gr.Label(num_top_classes = 1)).launch(show_error = True)